In [39]:
# Importations
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
import nltk
import time

from NLP_functions import clean_sentence, vader_scores, vectorize

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [22]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# Préparation des données

- minuscules 
- retirer ponctuation et nombres
- lemmatiser (wordNetLemmatizer)
- stopwords  
- tokenizer (nltk)


https://techvidvan.com/tutorials/python-sentiment-analysis/

In [5]:
df = pd.read_csv("finance_sentiment.csv", index_col=0)
df.head()

,Sentence,Sentiment,!,"""",#,$,%,&,(,),...,>,?,@,[,\,],_,`,|,~
0,The GeoSolutions technology will leverage Bene...,positive,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"For the last quarter of 2010 , Componenta 's n...",positive,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,According to the Finnish-Russian Chamber of Co...,neutral,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Swedish buyout firm has sold its remaining...,neutral,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# ajout de caractéristiques
df['clean_text'] = df.Sentence.apply(lambda text: clean_sentence(text))
df['vader_neg'] = df.clean_text.apply(lambda text: vader_scores(text)['neg'])
df['vader_neut'] = df.clean_text.apply(lambda text: vader_scores(text)['neu'])
df['vader_pos'] = df.clean_text.apply(lambda text: vader_scores(text)['pos'])
df['compound'] = df.clean_text.apply(lambda text: vader_scores(text)['compound'])
df.head()

,Sentence,Sentiment,!,"""",#,$,%,&,(,),...,],_,`,|,~,clean_text,vader_neg,vader_neut,vader_pos,compound
0,The GeoSolutions technology will leverage Bene...,positive,0,0,0,0,0,0,0,0,...,0,0,0,0,0,geosolutions technology leverage benefon gps s...,0.000,0.788,0.212,0.6249
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,0,0,0,3,0,0,0,0,...,0,0,0,0,0,esi low bk real possibility,0.344,0.656,0.000,-0.2732
2,"For the last quarter of 2010 , Componenta 's n...",positive,0,0,0,0,0,0,0,0,...,0,0,0,0,0,last quarter componenta net sale doubled eur e...,0.099,0.776,0.125,0.1531
3,According to the Finnish-Russian Chamber of Co...,neutral,0,0,0,0,0,0,0,0,...,0,0,0,0,0,according finnish russian chamber commerce maj...,0.000,1.000,0.000,0.0000
4,The Swedish buyout firm has sold its remaining...,neutral,0,0,0,0,0,0,0,0,...,0,0,0,0,0,swedish buyout firm sold remaining percent sta...,0.000,1.000,0.000,0.0000


In [7]:
# Vectorisation avec TfidVectorizer
df2 = vectorize(df)
print(df2.shape)
df2.head()

(5842, 9295)


,aa,aal,aaland,aalto,aaltonen,aapl,aaron,aava,aazhang,ab,...,zone,zoo,zp,zsl,zte,zu,zurich,zwe,zwhthsvfsf,zy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Création d'un dataframe contenant toutes les caractéristiques Sentiment, scores Vader, Ponctuation et vectorisation  

In [8]:
result =  pd.merge(df, df2, left_index = True, right_index = True)
result.head()

,Sentence,Sentiment,!,"""",#,$,%,&,(,),...,zone,zoo,zp,zsl,zte,zu,zurich,zwe,zwhthsvfsf,zy
0,The GeoSolutions technology will leverage Bene...,positive,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,0,0,0,3,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"For the last quarter of 2010 , Componenta 's n...",positive,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,According to the Finnish-Russian Chamber of Co...,neutral,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,The Swedish buyout firm has sold its remaining...,neutral,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
result.Sentiment.value_counts()

neutral     3130
positive    1852
negative     860
Name: Sentiment, dtype: int64

In [10]:
# Séparation des données en trainset et testset(0.2)
X_train, X_test, y_train, y_test = train_test_split(result.drop(['Sentence','Sentiment', 'clean_text'], axis=1), 
                                                    result.Sentiment, test_size=0.2, random_state=1234)

# Logistic Regression

In [32]:
logreg = LogisticRegression(max_iter=500, class_weight='balanced', random_state=123)
logreg.fit(X_train, y_train)
logreg.score(X_test, y_test)

0.7048759623609923

In [34]:
start = time.time()
log_pred = logreg.predict(X_test)
end = time.time()
lr_time = np.round(end-start,4)
print("Time elapsed: {} seconds".format(lr_time))
print(classification_report(y_test, log_pred))

Time elapsed: 0.0878 seconds
              precision    recall  f1-score   support

    negative       0.45      0.54      0.49       191
     neutral       0.79      0.74      0.77       620
    positive       0.73      0.73      0.73       358

    accuracy                           0.70      1169
   macro avg       0.66      0.67      0.66      1169
weighted avg       0.72      0.70      0.71      1169



In [56]:
# Dictionnaire résultats
lr_f1_neg, lr_f1_neut, lr_f1_pos = list(f1_score(y_test, log_pred, average=None))
lr_res = ['LogisticRegression', np.round(lr_f1_neg,2), np.round(lr_f1_neut,2),np.round(lr_f1_pos,2),lr_time]

In [20]:
# Avec over_sampling = augmentation du nb d'observations de classes minoritaires
rOs = RandomOverSampler()
X_ro, y_ro = rOs.fit_resample(X_train, y_train)
lr = LogisticRegression(max_iter=500, random_state=123)
lr.fit(X_ro, y_ro)
lr_pred = lr.predict(X_test)
print(classification_report(y_test, lr_pred))

              precision    recall  f1-score   support

    negative       0.44      0.50      0.47       191
     neutral       0.79      0.75      0.77       620
    positive       0.73      0.74      0.74       358

    accuracy                           0.70      1169
   macro avg       0.65      0.66      0.66      1169
weighted avg       0.71      0.70      0.71      1169



In [24]:
# Avec génération synthétique SMOTE
smo = SMOTE()
X_smo, y_smo = smo.fit_resample(X_train, y_train)
lr.fit(X_smo, y_smo)
y_pred_smo = lr.predict(X_test)
print(classification_report(y_test, y_pred_smo))

              precision    recall  f1-score   support

    negative       0.45      0.49      0.47       191
     neutral       0.79      0.75      0.77       620
    positive       0.73      0.75      0.74       358

    accuracy                           0.71      1169
   macro avg       0.66      0.67      0.66      1169
weighted avg       0.72      0.71      0.71      1169



# RandomForest

In [42]:
rf_clf = RandomForestClassifier(max_depth=250, class_weight='balanced', random_state=123)
rf_clf.fit(X_train, y_train)
rf_clf.score(X_test, y_test)

0.6612489307100086

In [43]:
start = time.time()
rf_pred = rf_clf.predict(X_test)
end = time.time()
rf_time = np.round(end-start,4)
print("Time elapsed: {} seconds".format(rf_time))
rf_f1_neg, rf_f1_neut, rf_f1_pos = list(f1_score(y_test, rf_pred, average=None))
print(classification_report(y_test, rf_pred))

Time elapsed: 0.0872 seconds
              precision    recall  f1-score   support

    negative       0.28      0.19      0.23       191
     neutral       0.69      0.80      0.74       620
    positive       0.75      0.68      0.71       358

    accuracy                           0.66      1169
   macro avg       0.57      0.56      0.56      1169
weighted avg       0.64      0.66      0.65      1169



In [57]:
# Dictionnaire résultats
rf_res = ['RandomForest', np.round(rf_f1_neg,2), np.round(rf_f1_neut,2),np.round(rf_f1_pos,2),rf_time]

# SVM

In [45]:
svm_clf = svm.SVC(kernel = 'linear', gamma = 0.1, class_weight='balanced')
svm_clf.fit(X_train, y_train)
svm_clf.score(X_test,y_test)

0.7134302822925578

In [47]:
start = time.time()
svm_pred = svm_clf.predict(X_test)
end = time.time()
svm_time = np.round(end-start,4)
print("Time elapsed: {} seconds".format(svm_time))
svm_f1_neg, svm_f1_neut, svm_f1_pos = list(f1_score(y_test, svm_pred, average=None))
print(classification_report(y_test, svm_pred))

Time elapsed: 38.8491 seconds
              precision    recall  f1-score   support

    negative       0.44      0.59      0.51       191
     neutral       0.82      0.73      0.77       620
    positive       0.74      0.75      0.74       358

    accuracy                           0.71      1169
   macro avg       0.67      0.69      0.67      1169
weighted avg       0.73      0.71      0.72      1169



In [58]:
# Dictionnaire résultats
svm_res = ['SVM', np.round(svm_f1_neg,2), np.round(svm_f1_neut,2),np.round(svm_f1_pos,2),svm_time]

# Naive Bayes

In [52]:
GNB_clf = GaussianNB()
GNB_clf.fit(X_train, y_train)
GNB_clf.score(X_test, y_test)

0.4833190761334474

In [53]:
start = time.time()
gnb_pred = GNB_clf.predict(X_test)
end = time.time()
gnb_time = np.round(end-start,4)
print("Time elapsed: {} seconds".format(gnb_time))
gnb_f1_neg, gnb_f1_neut, gnb_f1_pos = list(f1_score(y_test, gnb_pred, average=None))
print(classification_report(y_test, gnb_pred))

Time elapsed: 0.1266 seconds
              precision    recall  f1-score   support

    negative       0.24      0.39      0.29       191
     neutral       0.67      0.51      0.58       620
    positive       0.46      0.49      0.48       358

    accuracy                           0.48      1169
   macro avg       0.45      0.46      0.45      1169
weighted avg       0.53      0.48      0.50      1169



In [59]:
# Dictionnaire résultats
gnb_res = ['Naive Bayes',np.round(gnb_f1_neg,2),np.round(gnb_f1_neut,2),np.round(gnb_f1_pos,2),gnb_time]

# résultats

In [61]:
results_list = [lr_res, rf_res, svm_res, gnb_res]
df_results = pd.DataFrame(results_list, columns = ['Model', 'f1_negative', 'f1_neutral','f1_positive', 'time' ])
df_results

,Model,f1_negative,f1_neutral,f1_positive,time
0,LogisticRegression,0.49,0.77,0.73,0.0878
1,RandomForest,0.23,0.74,0.71,0.0872
2,SVM,0.51,0.77,0.74,38.8491
3,Naive Bayes,0.29,0.58,0.48,0.1266
